<a href="https://colab.research.google.com/github/krumeto/Data-Science-Study/blob/master/pytorch_09_dataloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader

import numpy as np
import math

In [3]:
!wget https://raw.githubusercontent.com/python-engineer/pytorchTutorial/master/data/wine/wine.csv

--2021-04-18 17:45:29--  https://raw.githubusercontent.com/python-engineer/pytorchTutorial/master/data/wine/wine.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10889 (11K) [text/plain]
Saving to: ‘wine.csv’

wine.csv            100%[===================>]  10.63K  --.-KB/s    in 0s      

2021-04-18 17:45:30 (93.6 MB/s) - ‘wine.csv’ saved [10889/10889]



In [4]:
class WineDataset(Dataset):

  def __init__(self):
    # data loading
    xy = np.loadtxt("./wine.csv", delimiter=",", dtype=np.float32, skiprows=1)
    self.x = torch.from_numpy(xy[:, 1:])
    self.y = torch.from_numpy(xy[:, [0]]) #n_samples, 1
    self.n_samples = xy.shape[0]


  def __getitem__(self, index):
    return self.x[index], self.y[index]

  def __len__(self):
    return self.n_samples


dataset = WineDataset()

In [6]:
first_data = dataset[0]
features, labels = first_data
print(features, labels)

tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
        3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
        1.0650e+03]) tensor([1.])


In [7]:
dataloader = DataLoader(dataset = dataset, batch_size=4, shuffle=True, num_workers=2)

dataiter = iter(dataloader)
data = dataiter.next()

features, labels = data
print(features, labels)

tensor([[1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
         3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
         1.0650e+03],
        [1.3620e+01, 4.9500e+00, 2.3500e+00, 2.0000e+01, 9.2000e+01, 2.0000e+00,
         8.0000e-01, 4.7000e-01, 1.0200e+00, 4.4000e+00, 9.1000e-01, 2.0500e+00,
         5.5000e+02],
        [1.3110e+01, 1.9000e+00, 2.7500e+00, 2.5500e+01, 1.1600e+02, 2.2000e+00,
         1.2800e+00, 2.6000e-01, 1.5600e+00, 7.1000e+00, 6.1000e-01, 1.3300e+00,
         4.2500e+02],
        [1.2850e+01, 3.2700e+00, 2.5800e+00, 2.2000e+01, 1.0600e+02, 1.6500e+00,
         6.0000e-01, 6.0000e-01, 9.6000e-01, 5.5800e+00, 8.7000e-01, 2.1100e+00,
         5.7000e+02]]) tensor([[1.],
        [3.],
        [3.],
        [3.]])


In [10]:
num_epochs = 2
total_samples = len(dataset)
n_iterations = math.ceil(total_samples/4)
print(total_samples, n_iterations)

for epoch in range(num_epochs):
  for i, (inputs, labels) in enumerate(dataloader):

    if (i+1) % 5 ==0:
      print(f"epoch {epoch+1}, step {i+1}, inputs {inputs.shape}")

178 45
epoch 1, step 5, inputs torch.Size([4, 13])
epoch 1, step 10, inputs torch.Size([4, 13])
epoch 1, step 15, inputs torch.Size([4, 13])
epoch 1, step 20, inputs torch.Size([4, 13])
epoch 1, step 25, inputs torch.Size([4, 13])
epoch 1, step 30, inputs torch.Size([4, 13])
epoch 1, step 35, inputs torch.Size([4, 13])
epoch 1, step 40, inputs torch.Size([4, 13])
epoch 1, step 45, inputs torch.Size([2, 13])
epoch 2, step 5, inputs torch.Size([4, 13])
epoch 2, step 10, inputs torch.Size([4, 13])
epoch 2, step 15, inputs torch.Size([4, 13])
epoch 2, step 20, inputs torch.Size([4, 13])
epoch 2, step 25, inputs torch.Size([4, 13])
epoch 2, step 30, inputs torch.Size([4, 13])
epoch 2, step 35, inputs torch.Size([4, 13])
epoch 2, step 40, inputs torch.Size([4, 13])
epoch 2, step 45, inputs torch.Size([2, 13])


## Transforms

In [22]:
class WineDataset(Dataset):

  def __init__(self, transform = None):
    # data loading
    xy = np.loadtxt("./wine.csv", delimiter=",", dtype=np.float32, skiprows=1)
    self.x = xy[:, 1:]
    self.y = xy[:, [0]] #n_samples, 1
    self.n_samples = xy.shape[0]

    self.transform = transform

  def __getitem__(self, index):
    sample = self.x[index], self.y[index]

    if self.transform:
      sample = self.transform(sample)

    return sample

  def __len__(self):
    return self.n_samples

class ToTensor:

  def __call__(self, sample):
    inputs, targets = sample
    return torch.from_numpy(inputs), torch.from_numpy(targets)

class MulTransform:

  def __init__(self, factor):
    self.factor = factor

  def __call__(self, sample):
    inputs, targets = sample
    inputs *= self.factor
    return inputs, targets

dataset = WineDataset(transform=ToTensor())

print(dataset[0])
print(type(dataset[0][0]))


composed = torchvision.transforms.Compose([ToTensor(), MulTransform(42)])
dataset = WineDataset(transform=composed)

print(dataset[0])
print(type(dataset[0][0]))

(tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
        3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
        1.0650e+03]), tensor([1.]))
<class 'torch.Tensor'>
(tensor([5.9766e+02, 7.1820e+01, 1.0206e+02, 6.5520e+02, 5.3340e+03, 1.1760e+02,
        1.2852e+02, 1.1760e+01, 9.6180e+01, 2.3688e+02, 4.3680e+01, 1.6464e+02,
        4.4730e+04]), tensor([1.]))
<class 'torch.Tensor'>
